In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import json

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import MDAnalysis as mda
from MDAnalysis.analysis.align import rotation_matrix
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis.rms import rmsd as rmsd_mdanalysis
import MDAnalysis.transformations as trans
from MDAnalysis.analysis.dihedrals import Ramachandran

import umap
import hdbscan


from Bio.PDB.DSSP import DSSP
from Bio.PDB import PDBParser

plt.style.use('default')

from dotenv import load_dotenv
load_dotenv()

import os

In [ ]:
b_dir = os.getenv('DATA_FOLDER')
b_dir

In [ ]:
# Peptides and simulation details
combined = True
title="WF1a_WF2"
peptide = "WF1a"
ref_pep = 1 # reference peptide
save_files = True


#Set start/end time and chop the peptide
start_sim = None 
stop_sim = None 
   
# To use C_alpha atoms only
c_alpha = True
step_size = 100

In [ ]:

folder_name = "last500ns"
base_folder = f"{b_dir}/umap_hdbscan/{folder_name}"
Path(base_folder).mkdir(parents=True, exist_ok=True)

save_files = True

if save_files:
    if combined:
        working_output_dir =f"{base_folder}/{peptide}"
        Path(working_output_dir).mkdir(parents=True, exist_ok=True)
    else:
        working_output_dir =f"{base_folder}"


In [ ]:
def calculate_distance(v1, v2):
    """
    Calculates the distances between two arrays
    """
    x1, y1, z1 = v1[0], v1[1], v1[2]
    x2, y2, z2 = v2[0], v2[1], v2[2]
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return distance

def unpack_lists(x):
    return_list = []
    for i in x:
        for e in i:
            return_list.append(e)
    return return_list

def get_distances_each_carbon_per_pep(u, frames, n_frames, amino_acid_count, 
                                      start_res, end_res, c_alpha_only=True):    

    if c_alpha_only:
        backbone = u.select_atoms(f"name CA and resid {start_res}:{end_res}")
    else:
        backbone = u.select_atoms(f"backbone and resid {start_res}:{end_res}")
        
    residues_range = amino_acid_count

    all_distances = np.full(
        (n_frames, int(len([(i, j) for i in range(residues_range) 
                            for j in range(residues_range) if i!=j])/2)
        ), fill_value=np.NaN
        )

    all_pos = np.full((n_frames, backbone.n_atoms * 3), fill_value=np.NaN)  

    # align to references structure (mean atom positions)

    for frame_index, ts in tqdm(enumerate(u.trajectory[frames]), total=n_frames):
        all_pos[frame_index] = backbone.positions.ravel()
        results = []
        i_already = []
        for i in range(backbone.n_atoms):
            i_already.append(i)
            for j in range(backbone.n_atoms):
                if i!=j and j not in i_already:
                    result = calculate_distance(
                        backbone.positions[i].ravel(), 
                        backbone.positions[j].ravel())
                    results.append(result)
        all_distances[frame_index] = results     

    return all_pos, all_distances

def get_dihedrals_individual_prot(u, start_id, end_id):

    u.trajectory[0]
    r = u.select_atoms(f"resid {start_id}-{end_id}")
    print(f"starting with residues {start_id} - {end_id}")
    R = Ramachandran(r).run(step=step_size)
    angles = R.results.angles
    phi = angles[:, :, 0]
    psi = angles[:, :, 1]
    combined_angles = np.dstack((phi, psi)).flatten()
    combined_angles = np.reshape(combined_angles, (phi.shape[0], phi.shape[1]*2))

    return combined_angles, phi, psi

In [ ]:
match_pep ={1:4, 2:3, 3:2, 4:1}

class Peptide:
    
    def __init__(self, xtc_file_path, tpr_file_path, peptide_number, 
                 peptide_order, amino_acid_count):
        self.xtc_file_path = xtc_file_path
        self.tpr_file_path = tpr_file_path
        self.peptide_number = peptide_number
        self.peptide_order = peptide_order
        self.amino_acid_count = amino_acid_count
        self.u = mda.Universe(tpr_file_path, xtc_file_path)
        self.u.trajectory.add_transformations(trans.unwrap(self.u.select_atoms(f"backbone")))
        if peptide_order == 1:
            self.pep_dict = {
                k: ((k-1)*amino_acid_count+1, k*amino_acid_count) for k in range(1, peptide_number+1)
            }
        else:
            self.pep_dict = {
                k: (len(self.u.select_atoms("protein").residues.resids)-match_pep[k]*amino_acid_count + 1, 
                    len(self.u.select_atoms("protein").residues.resids)-(match_pep[k]-1)*amino_acid_count) 
                    for k in range(1, peptide_number+1)
            }

        self.start_id = self.pep_dict[1][0]
        self.end_id = self.pep_dict[1][1]

    def get_pep_dict(self):
        return self.pep_dict
    
    def load_traj(self):
       
        start_sim, stop_sim, step = self.u.trajectory.check_slice_indices(None, None, None)

        frames = np.arange(start_sim, stop_sim, step_size)
        n_frames = frames.size

        frames_pd = pd.DataFrame(frames)
        frames_pd.to_csv(f"{working_output_dir}/frames.csv")

        return frames, n_frames
    

    def get_ref_pos(self, c_alpha_only=False):
        """
        Rotates the reference peptide according to the first position and then returns
        the mean structure positions
        """
        print(f"Starting analysis for peptide: {peptide}")
        print(f"Using reference pep: {ref_pep}, with start id: {self.start_id} and end_id: {self.end_id}")

        
        if c_alpha_only == True:
            backbone_ref = self.u.select_atoms(f"name CA and resid {self.start_id}:{self.end_id}")
        else:
            backbone_ref = self.u.select_atoms(f"resid {self.start_id}:{self.end_id}")
        self.u.trajectory[0]
        
        frames, n_frames = self.load_traj()
        first_pos_ref = backbone_ref.unwrap()

        all_pos1 = np.full((n_frames, backbone_ref.n_atoms, 3), fill_value=np.NaN)

        for frame_index, _ in tqdm(enumerate(self.u.trajectory[frames]), total=n_frames):
            backbone_ref.unwrap(inplace=True)
            backbone_ref.translate(-backbone_ref.center_of_geometry())
            R, rmsd = rotation_matrix(backbone_ref.positions, first_pos_ref)
            backbone_ref.rotate(R)
            all_pos1[frame_index] = backbone_ref.positions

        ref_pos = np.mean(all_pos1, axis=0)

        return ref_pos
    
    def get_distances(self):
        """
        Return the n-n distances between all the Calpha atoms in the WF1a peptides
        """

        frames, n_frames = self.load_traj()

        return [
            get_distances_each_carbon_per_pep(self.u, frames, n_frames, self.amino_acid_count, 
                                              resids[0], resids[1], c_alpha_only=True)[1]
            for k, resids in self.pep_dict.items()                                 
        ]
    
    def get_dihedrals(self):
        
        return [
            get_dihedrals_individual_prot(self.u,resids[0], resids[1], c_alpha_only=True)[1]
            for k, resids in self.pep_dict.items()                                 
        ]
    

In [ ]:
pep_1 = Peptide(
    f"/Volumes/miru_backup/jade_2synergy/popg/8_pg/WF1a_pg/md_0_1_combined.xtc",
    f"/Volumes/miru_backup/jade_2synergy/popg/8_pg/WF1a_pg/md_0_1.tpr",
    8, 1, 20 
)
pep_2 = Peptide(
    f"/Volumes/miru_backup/jade_2synergy/popg/8_pg/WF1a_pg_2/md_0_1_combined.xtc",
    f"/Volumes/miru_backup/jade_2synergy/popg/8_pg/WF1a_pg_2/md_0_1.tpr",
    8, 1, 20
)

WF1a_WF2_1 = Peptide(
    f"/Volumes/miru_backup/jade_2synergy/popg/WF1a_WF2_pg/md_0_1_combined.xtc",
    f"/Volumes/miru_backup/jade_2synergy/popg/WF1a_WF2_pg/md_0_1.tpr",
    4, 1, 20 
)

WF1a_WF2_2 = Peptide(
    f"/Volumes/miru_backup/jade_2synergy/popg_replicas/WF1a_WF2_pg_2/md_0_1_combined.xtc",
    f"/Volumes/miru_backup/jade_2synergy/popg_replicas/WF1a_WF2_pg_2/md_0_1.tpr",
    4, 1, 20 
)

In [ ]:
# Set simulation directories


pep_1 = Peptide(
    f"../WF1a_pg/md_0_1_combined.xtc",
    f"..//WF1a_pg/md_0_1.tpr",
    8, 1, 20 
)
pep_2 = Peptide(
    f"../WF1a_pg_2/md_0_1_combined.xtc",
    f"../WF1a_pg_2/md_0_1.tpr",
    8, 1, 20
)

WF1a_WF2_1 = Peptide(
    f"../WF1a_WF2_pg/md_0_1_combined.xtc",
    f"../WF1a_WF2_pg/md_0_1.tpr",
    4, 1, 20 
)

WF1a_WF2_2 = Peptide(
    f"..//WF1a_WF2_pg_2/md_0_1_combined.xtc",
    f"../WF1a_WF2_pg_2/md_0_1.tpr",
    4, 1, 20 
)



In [ ]:
#Calculate distances

distances_WF1a_WF2_1 = WF1a_WF2_1.get_distances()
distances_WF1a_WF2_2 = WF1a_WF2_2.get_distances()
distances_pep_1 = pep_1.get_distances()
distances_pep_2 = pep_2.get_distances()

### Normalize data

In [ ]:
#Combine all data

all_distances_sim1 = np.vstack(
    [
         *distances_WF1a_WF2_1, *distances_WF1a_WF2_2,
        *distances_pep_1, *distances_pep_2
        ]
    )

def get_normalized_data(data):

    max_values = np.amax(data, axis=0)
    normalized_data = data / max_values
    normalized_data = np.nan_to_num(normalized_data, nan=0)
    
    return normalized_data

### Remove low variance columns

In [ ]:
df_distances = pd.DataFrame(all_distances_sim1, columns=range(all_distances_sim1.shape[1]))

column_variance = df_distances.var()

plt.hist(column_variance.tolist())
plt.xlabel("Variance")
plt.ylabel("Frequency")
plt.title("Carbon alpha distances variance")
plt.show()

In [ ]:
to_drop = [i for i, c in enumerate(df_distances.var().tolist()) if c<10]
new_df = df_distances.drop(to_drop,axis=1)
new_df.to_csv(f"{working_output_dir}/distances_sim.csv")

hv_distances_sim1_2 = new_df.to_numpy()
normalized_hv_distances1_2 = get_normalized_data(hv_distances_sim1_2)
normalized_hv_distances1_2.shape


In [ ]:

with open(f'{working_output_dir}/normalised_distances_{peptide}.npy', 'wb') as f:
    np.save(f, normalized_hv_distances1_2)


In [ ]:
with open(f'{working_output_dir}/normalised_distances_{peptide}.npy', 'rb') as f:
    normalized_hv_dih_distances1_2 = np.load(f, allow_pickle=True)

normalized_hv_distances1_2 =normalized_hv_dih_distances1_2[:, :102]
normalized_hv_distances1_2.shape

### Run Umap

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

save_files = True
combined = True

params_dict = {"min_cluster_size": 70, "cluster_selection_epsilon": 1, "n_neighbours": 65, "random_state": 50,
 "start_chop": 0, "end_chop": 0}

with open(f'{working_output_dir}/hyper_params.json', 'w') as f:
    json.dump(params_dict, f)

X_embedded = umap.UMAP(
    n_neighbors=params_dict["n_neighbours"],  # larger numbers forcus more on global properties (and increase computational effort required)
    n_components=2,  # set the number of dimensions to reduce to
    min_dist=0.0,    # must be 0.0 if points are to be clustered later with HDBSCAN
    verbose=True,
    n_jobs=1,        # use all available cores
    random_state=params_dict["random_state"]
).fit_transform(normalized_hv_distances1_2)

hdb = hdbscan.HDBSCAN(
    min_cluster_size=params_dict["min_cluster_size"],              # minimum number of points for a group to be considered a cluster
    cluster_selection_epsilon=params_dict["cluster_selection_epsilon"],      # clusters closer than this distance apart will be merged
    cluster_selection_method="leaf"   # changes the way clusters are initialised. "eom" tends to lead to fewer clusters than "leaf"
)

hdb.fit(X_embedded)

y_pred = hdb.fit_predict(X_embedded)
y_pred = y_pred +1 


colors = [
    "#4f8c9d", "#72e5ef", "#bbc3fe", "#800f76",
    "#724363", "#1932bf", "#DC9D00", "#1f84ec", 
    "#8e3703", "#fbcab9", "#c00018", "#104b6d", "#fd7450", "#b38677",
    "#4f8c9d", "#72e5ef", "#104b6d", "#bbc3fe", "#800f76", "#ed8bc7",
    "#724363", "#ed2bb1", "#1932bf", "#8b6fed", "#1f84ec", 
    "#8e3703", "#fbcab9", "#c00018", "#fd7450", "#b38677"
    ]


c=[colors[i] for i in y_pred]

print(f"Not in a cluster: {sum(y_pred==-1)} ({100 * sum(y_pred==-1) / y_pred.size:.2f}%)")
outliers = X_embedded[y_pred==-1]

plt.figure(figsize=(6,4))

plt.scatter(X_embedded[:,0], X_embedded[:,1], cmap='Paired', s=30, c=c)
plt.scatter(outliers[:,0], outliers[:,1], cmap='Paired', s=30,c='black')

total_calc = 0
for cluster in range(-1, max(y_pred)+1):    
    mean = np.mean(X_embedded[y_pred==cluster], axis=0)
    total_calc += X_embedded[y_pred==cluster].shape[0]
    plt.text(
        mean[0]+0.85, mean[1],
        cluster,
        fontsize=15
    )


first_line_title =f" {peptide} peptides" if combined else peptide
plt.title(first_line_title,size=15,fontname="Times New Roman", fontweight="bold" )
plt.title(f"Outliers: {100 * sum(y_pred==0) / y_pred.size:.2f}%", size=12,fontname="Times New Roman", loc="right")
plt.ylabel("UMAP 2")
plt.xlabel("UMAP 1")
plt.savefig(f"{working_output_dir}/{peptide}_clusters.png", dpi=400, bbox_inches="tight")
cluster_labels = y_pred
plt.show()


### Process Clusters and Analyse Data

In [ ]:
df_labels = pd.DataFrame(cluster_labels)
df_labels["System"] = [
    *["WF1a + WF2 I" for i in cluster_labels[:251*4]],
    *["WF1a + WF2 II" for i in cluster_labels[251*4:251*8]],
    *["WF1a Pure I" for i in cluster_labels[251*8:251*16]],
    *["WF1a Pure II" for i in cluster_labels[251*16:251*24]]
    ]
df_labels["Time (ns)"] = [
    i%251 for i in range(len(cluster_labels))
]
df_labels["Peptide"] =  [
    *unpack_lists([[str(i) for x in range(251)] for i in range(1, 5)]),
    *unpack_lists([[str(i) for x in range(251)] for i in range(1, 5)]),
    *unpack_lists([[str(i) for x in range(251)] for i in range(1, 9)]),
    *unpack_lists([[str(i) for x in range(251)] for i in range(1, 9)]),
]
df_labels = df_labels.rename(columns={0: "Cluster"})
df_labels = df_labels[df_labels["Cluster"]!=0]

df_labels

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 2), sharex=True)

g = sns.stripplot(data=df_labels, y="System",
                x="Cluster",dodge=False, legend=True, size=15, jitter=True,c="#104b6d",
                 marker="o",linewidth=1, alpha=.2, ax=ax).set_title("WF1a (t>500ns)")
ax.set_xticks(range(1, len(np.unique(cluster_labels))+1))

plt.show()
fig.savefig(f"{working_output_dir}/all_clusters_per_system_{peptide}.png", dpi=400, bbox_inches="tight")


In [ ]:
pep_list_configs = {
    "WF1a": [
        "pep1", "pep2", "pep3", "pep4", "pep1_2", "pep2_2", "pep3_2", "pep4_2",
        "pep1_indiv", "pep2_indiv", "pep3_indiv", "pep4_indiv", 
        "pep5_indiv", "pep6_indiv", "pep7_indiv", "pep8_indiv", 
        "pep1_2_indiv", "pep2_2_indiv", "pep3_2_indiv", "pep4_2_indiv",
        "pep5_2_indiv", "pep6_2_indiv", "pep7_2_indiv", "pep8_2_indiv"
    ],
    "WF2": [
        "pep5", "pep6", "pep7", "pep8", "pep5_2", "pep6_2", "pep7_2", "pep8_2",
        "pep1_indiv", "pep2_indiv", "pep3_indiv", "pep4_indiv", 
        "pep5_indiv", "pep6_indiv", "pep7_indiv", "pep8_indiv", 
        "pep1_2_indiv", "pep2_2_indiv", "pep3_2_indiv", "pep4_2_indiv",
        "pep5_2_indiv", "pep6_2_indiv", "pep7_2_indiv", "pep8_2_indiv"
    ]
}

pep_list_names = pep_list_configs.get(peptide)

In [ ]:
#Split by peptide
n_frames = 251
cluster_labels_pep1 = cluster_labels[:n_frames]
cluster_labels_pep2 = cluster_labels[n_frames:2*n_frames]
cluster_labels_pep3 = cluster_labels[2*n_frames:3*n_frames]
cluster_labels_pep4 = cluster_labels[3*n_frames:4*n_frames]


#Combine the peptide dataframes
cluster_labels_time = pd.DataFrame([cluster_labels_pep1 , cluster_labels_pep2, cluster_labels_pep3, cluster_labels_pep4])
cluster_labels_time = cluster_labels_time.T
cluster_labels_time.columns = ["pep1", "pep2", "pep3", "pep4"] if peptide == "WF1a" \
      else ["pep5", "pep6", "pep7", "pep8"]

cluster_labels_reshaped = cluster_labels.reshape(n_frames,24)

cluster_labels_time = pd.DataFrame(cluster_labels_reshaped)

cluster_labels_time.columns = pep_list_names

#Save to file
cluster_labels_time.to_csv(f"{working_output_dir}/cluster_labels_{peptide}.csv")

frames_from_csv = pd.read_csv(f"{working_output_dir}/frames.csv")
frames_from_csv = frames_from_csv.drop(columns=["Unnamed: 0"])

cluster_frames_saved = frames_from_csv.join(cluster_labels_time)
cluster_frames_saved = cluster_frames_saved.rename(columns={"0":"Timeframe"})

### Cluster distribution per system

In [ ]:

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)


color_map = {
    i: colors[i] for i in range(-1, max(y_pred)+1)
}

def get_piechart(pep_list_names, subtitle):

    counter_dict = {x:0 for x in np.unique(cluster_labels)}
    total = 0
    for k, v in counter_dict.items():
        for pep in pep_list_names:
            val = cluster_labels_time[[pep]].value_counts().get(k)
            if val:
                counter_dict[k]+= val
                total += val
    counter_dict = {k:v for k, v in counter_dict.items() if k!=0}

    fig, ax = plt.subplots()
    ax.pie([v for k, v in counter_dict.items()], labels=counter_dict.keys(), colors=color_map,
            shadow=False, startangle=90, )
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig.gca().add_artist(centre_circle)

    # Equal aspect ratio ensures that pie is drawn as a circle
    ax.axis('equal')  
    plt.legend(labels=[f'{list(counter_dict.keys())[i]} {round(list(counter_dict.values())[i]/total*100)}%' 
    for i in range(len(counter_dict.keys()))], 
        bbox_to_anchor=(1,1))
    plt.title(f"{peptide} peptides - Cluster distribution \n {subtitle}",size=15,fontname="Times New Roman", fontweight="bold" )
    plt.tight_layout()
    # plt.savefig(f"{working_output_dir}/cluster_percentage_{subtitle}_{peptide}.png", dpi=400, bbox_inches="tight")
    plt.show()


In [ ]:
def get_piechart_with_arms(pep_list_names, subtitle):
    counter_dict = {x:0 for x in np.unique(cluster_labels)}
    total = 0
    for k, v in counter_dict.items():
        for pep in pep_list_names:
            val = cluster_labels_time[[pep]].value_counts().get(k)
            if val:
                counter_dict[k]+= val
                total += val
    counter_dict = {k:v for k, v in counter_dict.items() if k!=0}

    fig, ax = plt.subplots(figsize=(6,5))
    wedges, texts = ax.pie([v for k, v in counter_dict.items()], 
    labels=counter_dict.keys(), colors=[color_map[i] for i in list(counter_dict.keys())] ,
     wedgeprops=dict(width=0.5), startangle=-40, labeldistance=.58, textprops={'fontsize': 14})
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig.gca().add_artist(centre_circle)
    # Equal aspect ratio ensures that pie is drawn as a circle
    ax.axis('equal')  


    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.5)
    kw = dict(arrowprops=dict(arrowstyle="-"),
            bbox=bbox_props, zorder=0, va="center")
    
    total = sum([int(i) for i in counter_dict.values()])
    plt.tight_layout()
    # plt.savefig(f"{working_output_dir}/cluster_percentage_{subtitle}_{peptide}.png", bbox_inches="tight")
    plt.show()


In [ ]:

get_piechart_with_arms([p for p in pep_list_names if "indiv" not in p], "Combination Systems")               

In [ ]:
get_piechart_with_arms([p for p in pep_list_names if "indiv" in p], "Pure Systems")               

In [ ]:

def dssp_to_struct(dssp_entry):
        """
        Return the DSSP list associated with a dssp entry
        """
        sequence = ''
        sec_structure = ''
        for z in range(len(dssp_entry)):
                a_key = list(dssp_entry.keys())[z]
                sequence += dssp_entry[a_key][1]
                sec_structure += f"{dssp_entry[a_key][2]}"
        sec_structure.replace(' ', "C")
        
        return sec_structure    

In [ ]:
output_dir_pdb =f"{working_output_dir}/pdbs_{peptide}_per_pep"
Path(output_dir_pdb).mkdir(parents=True, exist_ok=True)

def write_pdb_per_pep(pep, pep_dict, u, frames, n_frames, ref_pos, ref_struct="min_rmsd"):
    """
    Find the representative structure for each cluster and save to .pdb file
    """
    start_id, end_id = pep_dict[pep]

    backbone = u.select_atoms(f"resid {start_id}:{end_id}")

    cluster_labels_pep = cluster_labels[(pep-1)*n_frames:pep*n_frames]


    list_peptide, list_cluster, list_first_frame, list_cluster_frames,\
          list_the_frame, cluster_indexes, list_dssp, all_frames_coordinates = \
            [], [], [], [], [], [], [], []
    

    for cluster in np.unique(cluster_labels_pep):
        
        # ref structures will be first structure in a cluster 
        first_frame_index = np.where(cluster_labels_pep==cluster)[0][0]
        first_frame = frames[first_frame_index]

        frames, n_frames =  pep_1.load_traj()

        # frames at which cluster is found
        frames_indexes = np.where(cluster_labels_pep==cluster)[0]
        frames_cluster = frames[frames_indexes]

        list_cluster_frames.append(frames_cluster)
        list_cluster.append(cluster)
        cluster_indexes.append(frames_indexes)
        list_peptide.append(pep)
        list_first_frame.append(first_frame)

        cluster_pos = np.full((len(frames_cluster), backbone.n_atoms, 3), fill_value=np.NaN)
        cluster_rmsd = np.full((len(frames_cluster), 1), fill_value=np.NaN)

        # align all and get positions
        frames_coordinates = []
        for frame_index, ts in enumerate(u.trajectory[frames_cluster]):
            backbone.translate(-backbone.center_of_geometry())
            R, rmsd_frame = rotation_matrix(backbone.positions, ref_pos)
            backbone.rotate(R)
            cluster_pos[frame_index] = backbone.positions
            cluster_rmsd[frame_index] = rmsd_frame
            frames_coordinates.append(u.trajectory.time)

        all_frames_coordinates.append(frames_coordinates)

        mean_rmsd = np.mean(cluster_rmsd)
        difference_array = np.absolute(cluster_rmsd - mean_rmsd)

        # find frame at which rmsd to mean position is smallest
        rep_frame_index = difference_array.argmin()

        rep_frame = frames_cluster[rep_frame_index]
    
        list_the_frame.append(rep_frame)

        # load coordinates from representative frame
        for frame_index, ts in enumerate(u.trajectory[[rep_frame]]):
            # translate and rotate aroms
            atoms = backbone
            atoms.unwrap(inplace=True)
            atoms.translate(-backbone.center_of_mass())

            R, rmsd = rotation_matrix(backbone.positions, ref_pos)
            atoms.rotate(R)

            # write pdb
            
            atoms.write(f"{output_dir_pdb}/{ref_struct}_cluster_{cluster}_pep{pep}.pdb")

            p = PDBParser()
            structure = p.get_structure(f"{cluster}_{pep}", f"{output_dir_pdb}/{ref_struct}_cluster_{cluster}_pep{pep}.pdb")
            model = structure[0]
            dssp = DSSP(model, f"{output_dir_pdb}/{ref_struct}_cluster_{cluster}_pep{pep}.pdb", dssp='mkdssp')
            list_dssp.append(dssp)

    return (list_peptide, list_cluster, list_first_frame, list_cluster_frames, list_the_frame, cluster_indexes, list_dssp, all_frames_coordinates)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

# pepdict_combined refers to the actual simulation datat
peptide_configs = {
    "WF1a": {
        1: (1, 20), 2: (21, 40), 3: (41, 60), 4: (61, 80), 5: (1, 20), 6: (21, 40),
        7: (41, 60), 8: (61, 80), 9: (1, 20), 10: (21, 40), 11: (41, 60), 12: (61, 80),
        13: (81, 100), 14: (101, 120), 15: (121, 140), 16: (141, 160), 17: (1, 20),
        18: (21, 40), 19: (41, 60), 20: (61, 80), 21: (81, 100), 22: (101, 120),
        23: (121, 140), 24: (141, 160)
    },
    "WF2": {
        1: (81, 105), 2: (106, 130), 3: (131, 155), 4: (156, 180), 5: (81, 105),
        6: (106, 130), 7: (131, 155), 8: (156, 180), 9: (1, 25), 10: (26, 50),
        11: (51, 75), 12: (76, 100), 13: (1, 25), 14: (26, 50), 15: (51, 75),
        16: (76, 100)
    }
}

pep_dict_combined = peptide_configs.get(peptide)

ref_pos = pep_1.get_ref_pos()

list_peptide, list_cluster, list_first_frame, list_cluster_frames, \
    list_the_frame, list_the_indexes, list_the_dssp, list_all_frames_coordinates = \
           [], [], [], [], [], [], [], []


for pep in tqdm(pep_dict_combined.keys()):
    if pep in range(1, 5):
        u = WF1a_WF2_1.u
    elif pep in range(4,9):
        u = WF1a_WF2_2.u
    elif pep in range(8, 17):
        u = pep_1.u
    elif pep in range(16, 25):
        u = pep_2.u
    else:
        u = None
    frames, n_frames =  pep_1.load_traj()

    list_peptide1, list_cluster1, list_first_frame1, list_cluster_frames1,\
         list_the_frame1, list_the_indexes1, list_the_dssp1, all_frames_coordinates1 = \
        write_pdb_per_pep(pep, pep_dict_combined, u, frames, n_frames, ref_pos=ref_pos)
        
    list_peptide.append(list_peptide1)
    list_cluster.append(list_cluster1)
    list_first_frame.append(list_first_frame1)
    list_cluster_frames.append(list_cluster_frames1)
    list_the_frame.append(list_the_frame1)
    list_the_indexes.append(list_the_indexes1)
    list_the_dssp.append(list_the_dssp1)
    list_all_frames_coordinates.append(all_frames_coordinates1)

structure_dataframe = pd.DataFrame([[unpack_lists(list_peptide)[k], unpack_lists(list_cluster)[k], 
                                unpack_lists(list_first_frame)[k], unpack_lists(list_cluster_frames)[k], 
                                unpack_lists(list_the_frame)[k], unpack_lists(list_the_indexes)[k], 
                                unpack_lists(list_the_dssp)[k], unpack_lists(list_all_frames_coordinates)[k]]
                                for k in range(len(unpack_lists(list_cluster)))],
                                columns=["peptide", "cluster", "first_frame", "cluster_frames", "the_frame", "indexes", "DSSP", "frames_cluster_coordinates"])
structure_dataframe["total_frames"] = structure_dataframe.apply(lambda x: len(x["cluster_frames"]), axis=1)
structure_dataframe["total_indexes"] = structure_dataframe.apply(lambda x: len(x["indexes"]), axis=1)
structure_dataframe["indexes"] = structure_dataframe["indexes"]+n_frames*(structure_dataframe["peptide"]-1)
structure_dataframe["DSSP"] = structure_dataframe.apply(lambda x: dssp_to_struct(x["DSSP"]), axis=1)
structure_dataframe_to_save = structure_dataframe
structure_dataframe_to_save["frames_cluster_coordinates"] = structure_dataframe_to_save.apply(lambda x: "_".join([str(x) for x in x["frames_cluster_coordinates"]]), axis=1)
structure_dataframe_to_save["cluster_frames"] = structure_dataframe_to_save.apply(lambda x: "_".join([str(x) for x in x["cluster_frames"].tolist()]), axis=1)
structure_dataframe_to_save.to_csv(f"{working_output_dir}/structure_dssp_frames_different_ref.csv")    
structure_dataframe["cluster_frames"] = structure_dataframe.apply(lambda x: [float(x) for x in x["cluster_frames"].split("_")], axis=1)
structure_dataframe["frames_cluster_coordinates"] = structure_dataframe.apply(lambda x: [float(x) for x in x["frames_cluster_coordinates"].split("_")], axis=1) 

# Convert the frames
all_frames = list(set(unpack_lists(structure_dataframe["cluster_frames"].tolist())))
all_frames = sorted(all_frames)
new_df_time_pep = pd.DataFrame(columns=all_frames, index=[1, 2, 3, 4])
for index, row in structure_dataframe.iterrows():
    for f in all_frames:
        if f in row["cluster_frames"]:
            new_df_time_pep.loc[row["peptide"], f] =  row["cluster"]
new_df_time_pep = new_df_time_pep[sorted(new_df_time_pep.columns.tolist())]
new_df_time_pep.to_csv(f"{working_output_dir}/clusters_time_frames_different_ref2.csv")   